In [1]:
import os
import numpy as np
import pandas as pd
from utils import *

In [2]:
agreement_results = {}
sent_level_agreement_results = {}
result_files, skip_sample_ids, selected_annotators, num_annotators = process_result_files()

label_maps = {
    'c-q-ub':{'consistent':2, 'questionable':1, 'unwanted':0, 'benign': 0},
    'c-q+b-u': {'consistent':2, 'questionable':1, 'benign': 1, 'unwanted':0},
    'c+b-q-u':{'consistent':2, 'benign': 2, 'questionable':1, 'unwanted':0},
    'c-b-u':{'consistent':2, 'benign': 1, 'questionable':np.nan, 'unwanted':0},
    'c+b-q+u':{'consistent':1, 'benign': 1, 'questionable':0, 'unwanted':0},
    'c-b+q+u':{'consistent':1, 'benign': 0, 'questionable':0, 'unwanted':0},
    'c-u':{'consistent':1, 'benign': np.nan, 'questionable':np.nan, 'unwanted':0},
}
for batch_id, file_path in enumerate(result_files):
    agreement_local_batch = {}
    sent_agreement_local_batch = {}
    for short_name, label_map in label_maps.items():
        agreement_num = compute_interannotator_agreement(file_path=file_path, label_map=label_map, selected_annotators=selected_annotators.get(file_path, None), skip_sample_ids=skip_sample_ids.get(file_path, []))
        agreement_local_batch[short_name] = agreement_num
        span_level_agreement_num = compute_span_level_interannotator_agreement(file_path=file_path, label_map=label_map, selected_annotators=selected_annotators.get(file_path, None), skip_sample_ids=skip_sample_ids.get(file_path, []))
        sent_agreement_local_batch[short_name] = span_level_agreement_num
        
    agreement_results[batch_id+1] = agreement_local_batch
    sent_level_agreement_results[batch_id+1] = sent_agreement_local_batch

annotator for agreement computation: ['forrest', 'renyi']
disagreed sample ids in batch: [14, 15, 16, 17, 19, 23]
forrest [0, 0, 0, 0, 0, 0]
renyi [2, 2, 2, 2, 2, 2]
interval Krippendorff's alpha for label map
{'consistent': 2, 'questionable': 1, 'unwanted': 0, 'benign': 0}
0.759
compute span level interannotator agreement
../assign/batch_5_src_no_sports/results/batch_1_annotation.json
annotator for span level agreement computation: ['forrest', 'renyi']
83
83
{'forrest': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1], 'renyi': [0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]}
disagreed sen

## Sample-level IAA

In [3]:
# pd display only 3 digits
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame.from_dict(agreement_results, orient='index')
df

,c-q-ub,c-q+b-u,c+b-q-u,c-b-u,c+b-q+u,c-b+q+u,c-u
1,0.759,0.858,0.870,0.858,0.841,0.759,0.908
2,0.610,0.665,0.593,0.724,0.600,0.639,0.824
3,0.571,0.596,0.474,0.622,0.417,0.569,0.666
4,0.660,0.616,0.524,0.643,0.520,0.656,0.671
5,0.493,0.499,0.429,0.613,0.308,0.396,0.686
6,0.645,0.707,0.663,0.732,0.603,0.625,0.790
7,0.878,0.653,0.309,0.747,0.320,0.891,1.000
8,0.761,0.778,0.778,0.844,0.824,0.861,0.844
9,0.441,0.563,0.564,0.598,0.522,0.382,0.683
10,0.409,0.611,0.636,0.672,0.434,0.338,0.762


In [4]:
df.mean()

c-q-ub    0.577
c-q+b-u   0.624
c+b-q-u   0.591
c-b-u     0.683
c+b-q+u   0.561
c-b+q+u   0.591
c-u       0.756
dtype: float64

## Sentence-level IAA

In [5]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.DataFrame.from_dict(sent_level_agreement_results, orient='index')
df

,c-q-ub,c-q+b-u,c+b-q-u,c-b-u,c+b-q+u,c-b+q+u,c-u
1,0.642,0.777,0.763,0.775,0.727,1.000,1.000
2,0.600,0.544,0.607,0.638,0.588,1.000,1.000
3,0.613,0.533,0.520,0.532,0.477,1.000,1.000
4,0.514,0.508,0.533,0.567,0.466,1.000,1.000
5,0.450,0.422,0.403,0.454,0.331,1.000,1.000
6,0.614,0.710,0.747,0.783,0.705,1.000,1.000
7,0.852,0.409,0.411,0.436,0.356,1.000,1.000
8,0.611,0.647,0.696,0.763,0.629,1.000,1.000
9,0.674,0.618,0.617,0.627,0.600,1.000,1.000
10,0.403,0.551,0.589,0.643,0.502,1.000,1.000
